In [1]:
%load_ext autoreload
%autoreload 3

In [2]:
import torch
import matplotlib.pyplot as plt
from flipper_training.configs import *
from flipper_training.utils.environment import make_x_y_grids, generate_heightmaps
from flipper_training.utils.heightmap_generators import MultiGaussianHeightmapGenerator
from flipper_training.vis.static_vis import plot_grids_xyz, plot_heightmap_3d, plot_single_heightmap

In [3]:
num_robots = 4

In [4]:
# Heightmap setup - use torch's XY indexing !!!!!
grid_res = 0.05  # 5cm per grid cell
max_coord = 6.4  # meters
x_grid, y_grid = make_x_y_grids(max_coord, grid_res, num_robots)

In [23]:
min_gaussians = 30
max_gaussians = 50
max_height_fraction = 0.1
min_std_fraction = 0.15  # 20% of maximum grid coordinate
max_std_fraction = 0.4  # 50% of maximum grid coordinate
min_sigma_ratio = 0.8  # maximum ratio of sigma x,y computed as smaller/larger
noise_std = 0.01
noise_mu = 0
heighmap_gen = MultiGaussianHeightmapGenerator(min_gaussians=min_gaussians, max_gaussians=max_gaussians, max_height_fraction=max_height_fraction, min_std_fraction=min_std_fraction, max_std_fraction=max_std_fraction, min_sigma_ratio=min_sigma_ratio, add_random_noise=True, noise_std=noise_std, noise_mu=noise_mu)

z_grid, suit_mask = generate_heightmaps(x_grid, y_grid, heighmap_gen)

In [ ]:
for HM_IDX in range(num_robots):
    plot_heightmap_3d(x_grid[HM_IDX], y_grid[HM_IDX], z_grid[HM_IDX]).show()

In [ ]:
robot_model = RobotModelConfig(robot_type="marv", voxel_size=0.08, points_per_driving_part=150)
world_config = WorldConfig(x_grid=x_grid, y_grid=y_grid, z_grid=z_grid, grid_res=grid_res, max_coord=max_coord, suitable_mask=suit_mask)

In [26]:
from flipper_training.utils.objective_managers import HeightAwareObjectiveManager

In [27]:
max_higher_allowed = 0.5
min_dist_to_goal = 5.
max_dist_to_goal = 10.
gen = HeightAwareObjectiveManager(higher_allowed=max_higher_allowed, min_dist_to_goal=min_dist_to_goal, max_dist_to_goal=max_dist_to_goal)

In [28]:
start_state, goal_state = gen.generate_start_goal_states(world_config, robot_model)

In [ ]:
for i in range(num_robots):
    ax = plot_single_heightmap(x_grid[i], y_grid[i], z_grid[i], start_state.x[i], goal_state.x[i])
    plt.show()

In [ ]:
for i in range(num_robots):
    f = plot_heightmap_3d(x_grid[i], y_grid[i], z_grid[i], start_state.x[i], goal_state.x[i])
    f.show()